In [1]:
!pip install dash

In [2]:
import json
import pandas as pd
import numpy as np
from datetime import date
import plotly.express as px
import plotly.graph_objects as go

import dash
from dash import html, dcc
from dash.dependencies import Input, Output

c:\Users\wuensche\Anaconda3\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


* **Aufgabe 1**
  * Für ein ausgewähltes Produkt Abhängigkeit von 'price_level' darstellen mit strip-plot
  * Für ein ausgewähltes Produkt Abhängigkeit von 'advertising' darstellen mit strip-plot
  * Für ein ausgewähltes Produkt Abhängigkeit von 'advertising' + 'price_level' darstellen mit strip-plot
* **Aufgabe 2**
  *  Für alle Produkte Abhängigkeit von 'advertising' darstellen

### Preprocessing

In [3]:
df = pd.read_csv("../../data/dunnhumby_BatF_transactions.csv")

df.WEEK_END_DATE = pd.to_datetime(df.WEEK_END_DATE, format="%d-%b-%y")

df = df[~df.PRICE.isnull()]

df['price_diff'] = df.BASE_PRICE - df.PRICE

df['price_direction'] = np.sign(df.price_diff)

df['price_level'] = df.price_direction.map({0: 'regular', -1: 'increase', 1: 'discounted'})

# Berechnung der Quantile für 'SPEND' innerhalb jeder 'STORE_NUM'
df['Umsatz_Quantil'] = df.groupby('STORE_NUM')['SPEND'].transform(
    lambda x: pd.qcut(x, 4, labels=['Q1', 'Q2', 'Q3', 'Q4'])
)


In [4]:
def plot_specProd_priceLvl(upc):
    fig = px.strip(df[df.UPC == upc], 
         y='PRICE', 
         x='WEEK_END_DATE', 
         color='price_level'
    )
    return fig

### Plot Funktionen (mit test)

In [5]:
# def plot_allupc():
#     fig = px.strip(
#         df_upc.set_index('marketing').loc[['N', 'T', 'D', 'F', 'FD']].reset_index(), 
#         y = 'relative_revenue', 
#         x = 'marketing',
#         #color = 'upc',
#         custom_data = ['upc'],
#         title = 'Effect per UPC'
#     )
#     return fig
# plot_allupc()

In [6]:
# def plot_upcstore(upc):
#     fig = px.strip(
#         df_upcstore[df_upcstore.upc == upc], 
#         y = 'relative_revenue', 
#         x = 'marketing',
#         #color = 'store_id',
#         category_orders = {'marketing': ['N', 'T', 'D', 'F', 'FD']},
#         title = f'Effect for upc {upc} per Store',
#         custom_data = ['upc', 'store_id']
#     )
#     return fig
    
# plot_upcstore(3800039118)

In [7]:
# def plot_upcstoreweek(upc, store_id):
#     fig = px.strip(
#         df_upcstoreweek[(df_upcstoreweek.UPC == upc) & (df_upcstoreweek.store_id == store_id)], 
#         y = 'relative_revenue', 
#         x = 'date', 
#         color = 'marketing',
#         title = f'Weekly UPC: {upc} @ {store_id}'
#     )
#     return fig
# plot_upcstoreweek(3800039118, 367)

In [8]:
# def create_default_figure():
#     # You can customize this as per your need. Here's an example:
#     fig = go.Figure()
#     fig.add_trace(go.Scatter(x=[], y=[])) # Empty Scatter Plot
#     fig.update_layout(
#         title="No data selected",
#         xaxis_title="X-axis",
#         yaxis_title="Y-axis"
#     )
#     return fig

# create_default_figure()

## Create Dashboard

In [9]:
app = dash.Dash(__name__)
app.layout = html.Div([
    html.H1("Marketing Effects (relative Revenue)"),

    # # Komponente zum Anzeigen der Callbackdaten
    # html.Div([  
    #     dcc.Markdown("""**Click Data**"""),
    #     html.Pre(id='jsonVis'), # Pre-formatted text element
    # ]),

    # # Hauptgrafik
    # dcc.Graph(
    #     id='main_graph',
    #     figure=plot_specProd_priceLvl(1600027527)
    # ),
    
    dcc.Dropdown(
        id='product-dropdown',
        options=[{'label': i, 'value': i} for i in df['UPC'].unique()],
        value=df['UPC'].unique()[0]  # Erstes Produkt als Standardwert
    ),

    dcc.DatePickerRange(
        id='date-picker-ranger',
        start_date=date(2009, 1, 14),
        end_date=date(2012, 1, 4),
        end_date_placeholder_text='Wählen Sie ein Datum aus!'
    ),

    dcc.Graph(id='revenue-time-series-graph'),
    
    dcc.Dropdown(
        id='quantile-dropdown',
        options=[{'label': q, 'value': q} for q in ['Q1', 'Q2', 'Q3', 'Q4']],
        value='Q1'  # Erstes Quantil als Standardwert
    ),
    dcc.Graph(id='marketing-impact-graph'),
    
    # # Zoomstufe 1
    # dcc.Graph(
    #     id='storeupc_graph'
    # ),

    # # Komponente zum Anzeigen der Callbackdaten
    # html.Div([
    #     dcc.Markdown("""**Click Data**"""),
    #     html.Pre(id='jsonVis2'), # Pre-formatted text element
    # ]),
    
    # # Zoomstufe 2
    # dcc.Graph(
    #     id='storeupcweek_graph'
    # ),
])

# Callbacks für interaktive Inhalte
# @app.callback(
#     Output(component_id='jsonVis', component_property='children'), # updates the text in the jsonVis component 
#     Input(component_id='main_graph', component_property='clickData') # whenever a point on the main_graph is clicked.
# )
# def display_click_data(clickData): #  It displays the click data in JSON format.
#     return json.dumps(clickData, indent=2)

@app.callback(
    Output('revenue-time-series-graph', 'figure'),
    [Input('product-dropdown', 'value')]
)
def update_time_series(selected_product):
    # Daten für das ausgewählte Produkt filtern
    filtered_df = df[df['UPC'] == selected_product]
    
    # Umsatz-Zeitreihendiagramm erstellen
    fig = px.line(filtered_df, x='WEEK_END_DATE', y='SPEND', title='Umsatz über die Zeit')
    return fig

@app.callback(
    Output('marketing-impact-graph', 'figure'),
    [Input('product-dropdown', 'value'),
     Input('quantile-dropdown', 'value')]
)
def update_marketing_impact(selected_product, selected_quantile):
    # Daten für das ausgewählte Produkt und Quantil filtern
    filtered_df = df[(df['UPC'] == selected_product) & (df['Umsatz_Quantil'] == selected_quantile)]
    
    # Berechnen des medianen Umsatzes ohne und mit Werbemaßnahmen
    median_spend_no_ad = filtered_df[filtered_df['FEATURE'] == 0]['SPEND'].median()
    median_spend_with_ad = filtered_df[filtered_df['FEATURE'] == 1]['SPEND'].median()
    
    # Bar-Graphen für den Vergleich der medianen Umsätze erstellen
    fig = px.bar(
        x=['Ohne Werbung', 'Mit Werbung'],
        y=[median_spend_no_ad, median_spend_with_ad],
        title='Impact der Werbemaßnahme auf den Umsatz'
    )
    return fig

# @app.callback(
#     Output(component_id='jsonVis2', component_property='children'), # updates the text in the jsonVis component 
#     Input(component_id='storeupc_graph', component_property='clickData') # whenever a point on the main_graph is clicked.
# )
# def display_click_data(clickData): #  It displays the click data in JSON format.
#     return json.dumps(clickData, indent=2)

# @app.callback(
#     Output(component_id='storeupc_graph', component_property='figure'), # updates the figure of the storeupc_graph
#     Input(component_id='main_graph', component_property='clickData') # whenever there is a click on a main_graph element
# )
# def update_fig2(clickData):
#     if clickData:
#         upc = clickData['points'][0]['customdata'][0]
#         return plot_upcstore(upc)
#     return create_default_figure()

# @app.callback(
#     Output(component_id='storeupcweek_graph', component_property='figure'),
#     Input(component_id='storeupc_graph', component_property='clickData'))
# def update_fig3(clickData):
#     if clickData:
#         upc = clickData['points'][0]['customdata'][0]
#         store_id = clickData['points'][0]['customdata'][1]
#         return plot_upcstoreweek(upc, store_id)
#     return create_default_figure()

SyntaxError: invalid syntax (2060969054.py, line 31)

Open Dashboard at `plot.{container_id}.laboration.hochschule-stralsund.de`
You find the container_id in the address of you browser

In [ ]:
app.run(jupyter_mode="tab", debug=True, host='localhost')

c:\Users\wuensche\Anaconda3\lib\site-packages\_plotly_utils\basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



Dash app running on http://localhost:8050/


<IPython.core.display.Javascript object>

c:\Users\wuensche\Anaconda3\lib\site-packages\_plotly_utils\basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

